In [2]:
library(kableExtra)
library(tidyverse)
library(fpp3)
library(tidyquant)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ purrr   1.0.1
✔ tibble  3.1.8     ✔ dplyr   1.1.0
✔ tidyr   1.2.1     ✔ stringr 1.4.1
✔ readr   2.1.4     ✔ forcats 1.0.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::group_rows() masks kableExtra::group_rows()
✖ dplyr::lag()        masks stats::lag()
── Attaching packages ────────────────────────────────────────────── fpp3 0.5 ──

✔ lubridate   1.9.2     ✔ feasts      0.3.0
✔ tsibble     1.1.3     ✔ fable       0.3.2
✔ tsibbledata 0.4.1     ✔ fabletools  0.3.2

── Conflicts ───────────────────────────────────────────────── fpp3_conflicts ──
✖ lubridate::date()    masks base::date()
✖ dplyr::filter()      masks stats::filter()
✖ dplyr::group_rows()  masks kableExtra::group_rows()
✖ tsibble::intersect() masks base::intersect()
✖ tsibble::interval()  masks lubridate::interval()
✖ dplyr::lag()       

## Univariate Models for Volatility
#### Motivating volatility time series models
Volatility is a fundamental variable in many areas of finance:
- Risk management
- Pricing of financial assets
- Hedging
- Option pricing
  * Univariate Models

In particular, we can: 
- model and forecast volatility (and conditional covariances/correlations, in the multivariate case)
- estimate more efficiently the parameters from the conditional mean equation (which establishes the model which plays the main role to obtain point forecasts);
- Establish correct point and interval forecasts for the variable of interest. In particular, if conditional heteroskedasticity is neglected, the prediction intervals will be incorrect.
#### Forecasting Time Series
- $R_{T+S|T}$ : foreacst you want to make s steps ahead obtained at time t for R (returns) at time t+s
- You want to make optimal forecasts, choose forecast that minimises MSE:
  - $min (MSE(R_{T+S|T})) = Var(\epsilon_{T+S|T}) = R_{T+S} - R_{T+S|T}$ ; s steps ahead forecast error
  - $R_{T+S|T} = E[R_{T+S} | I_T]$ ; Where its conditional on all the information you have 
  - The information set, I_T , includes the present and past information available and relevant to forecast
  - The relevant information available for the ARIMA models concerns the present and past values of the variable of interest Rt which implies
    - $\Rightarrow E[R_{T+S}|R_T, R_{T-1}, ...]$
    - For ADL models it also includes history of other variables revelant for returns
    - This formula then does not depend on volatlity $\sigma_t$
    - It playes a role in the prediction intervals, not for the point forecasts


#### Option Pricing
- Call Option: 
  - Contract that gives the right (but not the obligation) to buy a given financial asset at a fixed price until maturity (american) or at maturity date (european).
- Put Option:
  - Contract that gives the right (but not the obligation) to sell a given financial asset at a fixed price until maturity (american) or at maturity date (european).
- Black and Scholes (1972) and Merton (1973) extended the binomial model to continuous time and found the formula (BSM).
- From equation (1) we conclude that the price of financial options strongly depends on the asset volatility.
- In the BSM model, volatility is assumed constant which implies that it can be estimated by the historical standard deviation of returns

#### Portfolio Management
- Suppose that a financial practitioner wants to choose how much to invest in two different assets, say A e B, for the next periods. How to do it?
- A possibility is to build the efficient frontier. Here, he chooses the proportion to invest in asset A, $X_A$, and asset B, $X_B$, that minimizes the portfolio variance, $\sigma_P^2$, and achieves a fixed target expected return, $\mu_P$
- $P = X_A.A + X_B.B = X_A.A + (1-X_A).B$
- $min X_A Var(P) = X_A^2 . Var(A) + X_B^2.Var(B) + 2.X_A.X_B\sigma_{A,B} ;  s.t E(P) = \bar{\mu_p} = E[X_A.A + X_B.B]$
- 

In [4]:
stock <- tidyquant::tq_get(c("^GSPC","^N225","^IXIC","^NYA"), 
   get = "stock.prices" ,from="1990-01-01", to="2021-12-31") %>% 
  mutate(symbol = ifelse(symbol == "^GSPC","S&P500",symbol)) %>% 
  mutate(symbol = ifelse(symbol == "^N225","Nikkei",symbol)) %>% 
  mutate(symbol = ifelse(symbol == "^IXIC","NASDAQ",symbol)) %>% 
  mutate(symbol = ifelse(symbol == "^NYA","NYSE",symbol)) %>% 
  mutate(date = as_date(date)) %>% 
  as_tsibble(index = date,key = symbol) %>% 
  group_by_key() %>% 
  mutate(rtn = difference(log(close))) %>% 
  filter(!is.na(rtn)) %>% 
  group_by_key() %>% 
  mutate(trading_day = row_number()) 
stock %>% select(symbol,date,close,rtn)

symbol,date,close,rtn
<chr>,<date>,<dbl>,<dbl>
NASDAQ,1990-01-03,460.9,0.003477522
NASDAQ,1990-01-04,459.4,-0.003259810
NASDAQ,1990-01-05,458.2,-0.002615481
NASDAQ,1990-01-08,458.7,0.001090632
NASDAQ,1990-01-09,456.8,-0.004150796
NASDAQ,1990-01-10,450.7,-0.013443676
NASDAQ,1990-01-11,448.9,-0.004001824
NASDAQ,1990-01-12,439.7,-0.020707429
NASDAQ,1990-01-15,436.6,-0.007075246


#### Structure Of Model
- Decompose R_t as:
  - $r_t = E(r_t | I{t-1} + R_t - E(r_t | I{t-1}= E(r_t | I{t-1}) + \epsilon_t = \mu_t + \epsilon_t \Rightarrow$ Conditional Mean Equation
  - Until now, epsilon has been regarded as iid with mean 0, variance sigma^2
    - Now it will be defined as a product of two RV
    - 